In [8]:
import chromadb

client = chromadb.PersistentClient(path="./chroma_db")

In [9]:
COLLECTION_NAME = "lyric_embeddings"
collection = client.get_collection(name=COLLECTION_NAME)


In [ ]:
collection.count()  # Returns the number of documents in the collection

In [ ]:
collection._embedding_function

In [14]:
song_lryics = """
Nice to meet you
Where you been?
I could show you incredible things
Magic, madness, heaven, sins
Saw you there and I thought oh my god
Look at that face, you look like my next mistake

Love's a game, wanna play?

New money, suit and tie

I can read you like a magazine

Ain't it funny rumors fly

And I know you heard about me

So hey, let's be friends

I'm dying to see how this one ends

Grab your passport and my hand

I can make the bad guys good for a weekend



So it's gonna be forever

Or it's gonna go down in flames

You can tell me when it's over

If the high was worth the pain

Got a long list of ex-lovers

They'll tell you I'm insane

Cause you know I love the players

And you love the game



Cause we're young and we're reckless

We'll take this way too far, It'll leave you breathless

Or with a nasty scar

Got a long list of ex-lovers

They'll tell you I'm insane

But I've got a blank space baby

And I'll write your name



Cherry lips

Crystal skies

I could show you incredible things

Stolen kisses, pretty lies 

You're the king baby I'm your queen

Find out what you want

Be that girl for a month

Wait, the worst is yet to come

Oh no

Screaming, crying, perfect storms

I can make all the tables turn

Rose garden filled with thorns

Keep you second guessing like, oh my god

Who is she? I get drunk on jealousy

But you'll come back each time you leave

Cause darling I'm a nightmare dressed like a daydream



So it's gonna be forever

Or it's gonna go down in flames

You can tell me when it's over

If the high was worth the pain

Got a long list of ex-lovers

They'll tell you I'm insane

Cause you know I love the players

And you love the game



Cause we're young and we're reckless

We'll take this way too far, It'll leave you breathless

Or with a nasty scar

Got a long list of ex-lovers

They'll tell you I'm insane

But I've got a blank space baby

And I'll write your name



Boys only want love

If it's torture

Don't say I didn't say I didn't warn you

Boys only want love, If it's torture

Don't say I didn't say I didn't warn you



So it's gonna be forever

Or it's gonna go down in flames

You can tell me when it's over

If the high was worth the pain

Got a long list of ex-lovers

They'll tell you I'm insane

Cause you know I love the players

And you love the game



Cause we're young and we're reckless

We'll take this way too far, It'll leave you breathless

Or with a nasty scar

Got a long list of ex-lovers

They'll tell you I'm insane

But I've got a blank space baby

And I'll write your name
"""

In [ ]:
# Search for ed sheeran song using chroma query which is part of metadata object with artist name - Ed Sheeran
# Print only the title of the song which is part of the metadata object
results = collection.query(
    query_texts=[song_lryics],
    n_results=5
)

for result in results["metadatas"][0]:
    print(result)  # Assuming 'title' is the key for the song title in metadata

In [ ]:
%pip install rapidfuzz

In [ ]:
results["documents"][0]

In [ ]:
# Get lyrics from the results and do a fuzzy match with the song_lryics
from rapidfuzz import fuzz

# Get lyrics from the results
lyrics = results["documents"][0]

# Do a fuzzy match with the song_lryics
similarity = fuzz.token_ratio(lyrics[1], song_lryics.lower())
similarity


In [16]:
import pandas as pd
import numpy as np

df = pd.read_csv("data/spotify_audio_features.csv")


In [ ]:
df.head()

In [ ]:
%%time
print("\n--- Starting metadata update loop ---")
for index, row in df.iterrows():

    track_artist = row["track_artist"]
    track_name = row["track_name"]
    try:
        result = collection.query(
            query_texts=[track_name],
            n_results=1,
            where={"artist": track_artist},
        )
        audio_features = row.iloc[4:].to_dict()

        if result["metadatas"] and result["metadatas"][0] and result["ids"] and result["ids"][0]:
            retrieved_id = result["ids"][0]
            if isinstance(retrieved_id, list):
                retrieved_id = retrieved_id[0]

            current_metadata = result["metadatas"][0]
            if isinstance(current_metadata, list):
                current_metadata = current_metadata[0]

            # Flatten audio_features directly into current_metadata
            current_metadata.update({
                "track_artist": track_artist,
                "track_name": track_name,
                **audio_features
            })

            collection.update(
                ids=[retrieved_id],
                metadatas=[current_metadata]
            )
            print(f"  Index: {retrieved_id} Successfully updated metadata for '{track_name}' by '{track_artist}'.")
        else:
            print(f"  No matching document found or invalid result structure for '{track_name}' by '{track_artist}'. Skipping update.")

    except Exception as e:
        # Catch any exception during the query or update and print it
        print(f"  ERROR processing '{track_name}' by '{track_artist}': {e}")
        continue

In [12]:
results = collection.query(
    query_texts=["""Nice to meet you
Where you been?
I could show you incredible things
Magic, madness, heaven, sins
Saw you there and I thought oh my god
Look at that face, you look like my next mistake

Love's a game, wanna play?

New money, suit and tie

I can read you like a magazine

Ain't it funny rumors fly

And I know you heard about me

So hey, let's be friends

I'm dying to see how this one ends

Grab your passport and my hand

I can make the bad guys good for a weekend



So it's gonna be forever

Or it's gonna go down in flames

You can tell me when it's over

If the high was worth the pain

Got a long list of ex-lovers

They'll tell you I'm insane

Cause you know I love the players

And you love the game



Cause we're young and we're reckless

We'll take this way too far, It'll leave you breathless

Or with a nasty scar

Got a long list of ex-lovers

They'll tell you I'm insane

But I've got a blank space baby

And I'll write your name



Cherry lips

Crystal skies

I could show you incredible things

Stolen kisses, pretty lies 

You're the king baby I'm your queen

Find out what you want

Be that girl for a month

Wait, the worst is yet to come

Oh no

Screaming, crying, perfect storms

I can make all the tables turn

Rose garden filled with thorns

Keep you second guessing like, oh my god

Who is she? I get drunk on jealousy

But you'll come back each time you leave

Cause darling I'm a nightmare dressed like a daydream



So it's gonna be forever

Or it's gonna go down in flames

You can tell me when it's over

If the high was worth the pain

Got a long list of ex-lovers

They'll tell you I'm insane

Cause you know I love the players

And you love the game



Cause we're young and we're reckless

We'll take this way too far, It'll leave you breathless

Or with a nasty scar

Got a long list of ex-lovers

They'll tell you I'm insane

But I've got a blank space baby

And I'll write your name



Boys only want love

If it's torture

Don't say I didn't say I didn't warn you

Boys only want love, If it's torture

Don't say I didn't say I didn't warn you



So it's gonna be forever

Or it's gonna go down in flames

You can tell me when it's over

If the high was worth the pain

Got a long list of ex-lovers

They'll tell you I'm insane

Cause you know I love the players

And you love the game



Cause we're young and we're reckless

We'll take this way too far, It'll leave you breathless

Or with a nasty scar

Got a long list of ex-lovers

They'll tell you I'm insane

But I've got a blank space baby

And I'll write your name"""],
    n_results=5,
    where={"track_name": "I Don't Care (with Justin Bieber) - Loud Luxury Remix"}
)

for result in results["metadatas"][0]:
    print(result)  # Assuming 'title' is the key for the song title in metadata